'''


    @Author: Prathamesh Deshpande
    @Date: 21/11/2024
    @Last Modified by: Prathamesh Deshpande
    @Last Modified time: 21/11/2024
    @Title: Python code to perform infering(classify the reviews as postive,negative or neutral) and expanding(Process review to generate more info.) using Gemini AI.


'''

Importing Libraries

In [1]:
import os
import google.generativeai as genai
import csv
import time

Configure the Gemini AI

In [2]:
genai.configure(api_key=os.getenv('GEMINI_API_KEY'))

In [3]:
generation_config = {
  "temperature": 0.9,
  "top_p": 0.95,
  "top_k": 64,
  "max_output_tokens": 8192,
}

model = genai.GenerativeModel(
  model_name="gemini-1.5-flash",
  generation_config=generation_config,)

Reading Reviews from Product_Reviews.txt file

In [4]:
def read_reviews(file_path, delimiter="---END OF REVIEW---"):
    """
    Description: 
        This function reads the c from the text file

    Parameters:
        file_path: Path of txt file
        delimiter: delimiter

    Return:
        list: list of reviews
    """

    with open(file_path, 'r') as file:
            reviews = file.read().split(delimiter)
            
    return [review.strip() for review in reviews if review.strip()]

In [5]:
def extract_review_info(review):
    """
    Description: 
        This function extracts product name, and body from review

    Parameters:
        review: review body

    Return:
        product, review_text (str)
    """

    lines = review.split("\n")
    product = ""
    review_text = ""

    for line in lines:
        if line.startswith("Product:"):
            product = line.split(":")[1].strip()
            
        elif line.startswith("Review:"):
            review_text = line.split(":")[1].strip()
    
    return product, review_text

In [ ]:
def analyze_sentiment(review_text, chat_session):
    """
    Description: 
        This function analyze review sentiment and gives proper reply
    Parameters:
        review_text: review body
        chat_session: gemini ai chat session
    Return:
        sentiment, reply (str)
    """
    
    # Ask Gemini to categorize the sentiment of the review
    sentiment_response = chat_session.send_message(f"Categorize the sentiment in one word of this review in Postive/Negative/Neutral: {review_text}")
    sentiment = sentiment_response.text.lower()

    # Determine reply based on sentiment
    reply = chat_session.send_message(f"Add a 40 words reply to this review: {review_text} as per sentiment {sentiment}")
    reply = reply.text

    return sentiment, reply

In [7]:
def guess_product(review_text, chat_session):
    """
    Description: 
        This function guesses the product category based on the review text
    Parameters:
        review_text: review body
        chat_session: gemini ai chat session
    Return:
        product_response (str)
    """
    
    # Ask Gemini to guess the product category based on the review text
    product_response = chat_session.send_message(f"Guess the product name based on this review in one word: {review_text}")
    return product_response.text.strip()

Save Output to csv file

In [8]:
def save_to_csv(data, csv_file):
    """
    Description: 
        This function save the response in csv file
        
    Parameters:
        data: data to be saved in csv
        csv_file: csv file name/path
    """
    with open(csv_file, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["Original Product", "Guessed Product", "Review", "Sentiment", "Reply"])  # CSV header
        writer.writerows(data)

In [ ]:
def process_reviews(file_path, csv_file, chat_session, delimiter="---END OF REVIEW---"):
    """
    Description: 
        This Function processes the review
        
    Parameters:
        file_path: file path of txt file
        csv_file: csv file path
        chat_session: gemini ai chat session 
    """
    reviews = read_reviews(file_path, delimiter)
    data = []

    for review in reviews:
        original_product, review_text = extract_review_info(review)
        guessed_product = guess_product(review_text, chat_session)
        sentiment, reply = analyze_sentiment(review_text, chat_session)
        data.append([original_product, guessed_product, review_text, sentiment, reply])
        time.sleep(10)
    
    save_to_csv(data, csv_file)

In [10]:
def main():
    chat_session = model.start_chat(
        history=[]
    )
    process_reviews("Product_Reviews.txt","Processed_Reviews.csv",chat_session)

In [11]:
if __name__ == '__main__':
    main()